Welcome to my kernel, I hope you'll find it useful as a brief tutorial. It has two steps: data preprocessing, where I modify the datasets to achieve the best performance of the model, and building the model itself. 

![Photo from Pixabay](https://cdn.pixabay.com/photo/2016/01/19/17/08/vintage-1149558_1280.jpg)

# 1-Data preprocessing 🛠

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

routeTrain='../input/house-prices-advanced-regression-techniques/train.csv'
routeTest='../input/house-prices-advanced-regression-techniques/test.csv'


datasetTrain=pd.read_csv(routeTrain)
datasetTest=pd.read_csv(routeTest)  



In [ ]:
datasetTrain.info()

In [ ]:
datasetTest.info()

Drop the columns with a lot of NaN values **in both datasets**. These colums are MiscFeature, PoolQC, Fence, FireplaceQu,Alley.

In [ ]:
datasetTrain=datasetTrain.drop(['MiscFeature','PoolQC','Fence','FireplaceQu','Alley'],axis=1)
datasetTest=datasetTest.drop(['MiscFeature','PoolQC','Fence','FireplaceQu','Alley'],axis=1)

Build a heatmap to check which variables are more correlated to SalePrice, the variable that you want to predict. The heatmap will show only the numerical variables. To study the correlation between the target variable and a categorical variable, you can do the ANOVA test. 

In [ ]:
corr=datasetTrain.corr()
plt.figure(figsize = (40,40))
sns.heatmap(corr,annot=True)

In [ ]:
cor_target = abs(corr["SalePrice"])#Selecting highly correlated features
important_numerical_features = cor_target[cor_target>0.5]
important_numerical_features

The numerical variables that are correlated to SalePrice (correlation >=0.5) are:

* OverallQual: positive correlation, which means that SalePrice is bigger when the OverallQual is bigger.
* YearBuilt: positive correlation.
* YearRemodAdd: positive correlation.
* TotalBsmtSF: positive correlation.
* 1stFlrSF: positive correlation.
* GrLivArea: positive correlation.
* FullBath: positive correlation.
* TotRmsAbvGrd: positive correlation.
* GarageCars: positive correlation.
* GarageArea: positive correlation.


In [ ]:
#if you want to check the correlation between categorical variables and target variable
#Example-MSZoning
from scipy import stats
F, p = stats.f_oneway(datasetTrain[datasetTrain.MSZoning=='RL'].SalePrice,datasetTrain[datasetTrain.MSZoning=='RM'].SalePrice,datasetTrain[datasetTrain.MSZoning=='C (all)'].SalePrice,
                     datasetTrain[datasetTrain.MSZoning=='FV'].SalePrice,datasetTrain[datasetTrain.MSZoning=='RH'].SalePrice)
print(F)

The next step is to check if any of the numerical variables are right or left skewed. You can do this with the function skew(). In a normal distribution, the value of skewness is zero. When a distribution is asymmetrical the tail of the distribution is skewed to one side-to the right (value of the skewness is positive) or to the left (skewness is negative). To fix skewed variables, use log transformation.

In [ ]:
numerical_columns=['OverallQual','YearBuilt','YearRemodAdd','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','TotRmsAbvGrd','GarageCars','GarageArea','SalePrice']

In [ ]:
datasetTrain[numerical_columns].skew()

In [ ]:
fig = px.histogram(datasetTrain, x="GrLivArea")
fig.show()

You can clearly see that GrLivArea is right skewed. 

In [ ]:
fig = px.histogram(datasetTrain, x="TotalBsmtSF")
fig.show()

In [ ]:
fig = px.histogram(datasetTrain, x="GarageArea")
fig.show()

However, with the variables TotalBsmtSF and GarageArea there is a problem: when the house has not basement or garage, the value for these column in that specific row is 0. So if you apply log transformation to these two, you'll get a weird result. 

The solution I came up with was to throw away these rows.

In [ ]:
housesWithNoBasement=datasetTrain[datasetTrain.TotalBsmtSF==0]
housesWithNoBasement.shape #rows,columns

In [ ]:
housesWithNoGarage=datasetTrain[datasetTrain.GarageArea==0]
housesWithNoGarage.shape

There were 1460 not null values in TotalBsmtSF and GarageArea columns so it's not big deal deleting 81 and 37 rows.

In [ ]:
datasetTrain=datasetTrain[datasetTrain.TotalBsmtSF>0]
datasetTrain=datasetTrain[datasetTrain.GarageArea>0]

Let's select now the columns that had a significant correlation with the target variable, and SalePrice itself, and let's apply log transformation to them. 

In [ ]:
finalDatasetTrain=datasetTrain[numerical_columns]
finalDatasetTrain=np.log1p(finalDatasetTrain)

In [ ]:
finalDatasetTrain.skew()

In [ ]:
finalDatasetTrain.info()

Let's prepare the test dataset.

In [ ]:
nun_columns=['OverallQual','YearBuilt','YearRemodAdd','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','TotRmsAbvGrd','GarageCars','GarageArea']
finalTestDataset=datasetTest[nun_columns]

In [ ]:
finalTestDataset.info()

In [ ]:
finalTestDataset.GarageArea.unique()

In [ ]:
finalTestDataset.GarageCars.unique()

In [ ]:
finalTestDataset.update(finalTestDataset['GarageCars'].fillna(value=finalTestDataset['GarageCars'].mean(), inplace=True))

In [ ]:
finalTestDataset.update(finalTestDataset['GarageArea'].fillna(value=finalTestDataset['GarageArea'].mean(), inplace=True))

In [ ]:
finalTestDataset.info()

In [ ]:
finalTestDataset=np.log1p(finalTestDataset)

# 2-Building the model 🏛

In [ ]:
#build a basic model for the tutorial
import xgboost as xgb
model=xgb.XGBRegressor(max_depth=3,eta=0.05,min_child_weight=4)


In [ ]:
features=finalDatasetTrain.drop('SalePrice',axis=1)
y=finalDatasetTrain['SalePrice']

In [ ]:
model.fit(features,y)
predictions=model.predict(finalTestDataset)

In [ ]:
finalPred=np.expm1(predictions)#you need to reverse log transformation



In [ ]:
sample_submission=pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
submission = pd.DataFrame({'Id':sample_submission['Id'],'SalePrice':finalPred})
submission


In [ ]:
filename = 'Submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)